In [5]:
import matplotlib.pyplot as plt
import numpy as np
%load_ext autoreload
%autoreload 2
%matplotlib qt

$
\newcommand{\set}[2]{\big\{#1\,\ {\large:}\ \,#2\big\}}
\newcommand{\eps}{\varepsilon}
\newcommand{\abs}[1]{\left\vert#1\right\vert}
\newcommand{\ceil}[1]{\left\lceil#1\right\rceil}
\newcommand{\floor}[1]{\left\lfloor#1\right\rfloor}
\newcommand{\sinal}{\operatorname{sinal}}
$
# O método da bissecção

## $ \S 1 $ Introdução

O método da bissecção é um procedimento geral para determinação de zeros de uma
função  real $ f $. Seus únicos requerimentos são que a função seja contínua e
que tenhamos previamente identificado um intervalo $ [a, b] $ dentro do domínio
de $ f $ onde ela troca de sinal. Seus principais méritos são:
* _Confiabilidade_: ele sempre converge a um zero, satisfeitas as hipóteses
* acima.
* _Simplicidade_: ele é facilmente implementado e não é necessário calcular a
* derivada da função.
* _Robustez_: pequenos erros de arredondamento (comparados à precisão desejada)
* não impactam seu resultado.

Por outro lado, o método não pode ser empregado para se localizar um zero de
multiplicidade par (como o da função $ x \mapsto x^2 $). Além disto, em muitos
casos é possível utilizar métodos cuja convergência é ainda mais rápida. Se
queremos obter um zero de uma única função, esta diferença de desempenho não é
relevante, porém a situação muda se o algoritmo precisa ser executado bilhões ou
trilhões de vezes.

## $ \S 2 $ Descrição do método da bissecção

Como acima, sejam $ f $ uma função real _contínua_ e $ [a, b] $ um intervalo dentro do domínio de $ f $ com
$$
\boxed{\sinal f(a) \ne \operatorname{sinal} f(b)}
$$
No **método da bissecção** começamos tomando $ m $ como o **ponto médio** de $ a $ e $ b $,
$$
\boxed{m = \frac{a+b}{2}} 
$$
e avaliando $ f $ aí. Há três possibilidades:
* Se $ f(m)= 0 $, então $ m $ é um zero de $ f $ e podemos terminar.
* Se $ \sinal f(m)  \neq \sinal f(a) $, então $ f $ troca de sinal em $ [a, m] $.
* Se $ \sinal f(m) = \sinal f(a) $, então $ f $ troca de sinal em $ [m, b] $.

Nos dois últimos casos, podemos restringir nossa busca a um intervalo cujo comprimento é a *metade* do anterior. O teorema do valor intermediário garante que este subintervalo ainda contém um zero, portanto podemos repetir o processo. Bissecções sucessivas eventualmente nos levarão a um zero ou a um encaixotamento de um zero por um subintervalo de comprimento tão pequeno quanto desejado. Quando isto acontecer, retornamos o ponto médio deste subintervalo como estimativa para o zero.

📝 O método da bissecção também é conhecido como _busca binária_.

📝 Não é necessário _isolar_ um zero dentro do intervalo inicial, ou seja, o método funciona mesmo que haja mais de um zero dentro de $ [a, b] $.

**Exemplo 1:** Execute em seqüência as três células abaixo para ver uma animação do método da bissecção aproximando um zero da função $ f(x) = x^3 - x - 3 $ (em $ 1.6717 $, aproximadamente).

In [6]:
def print_solution(xs, ys, freq=1):
    """
    Given two arrays 'xs' and 'ys' of the same length, prints a table whose n-th
    line consists of three entries: the values of n, xs[n] and ys[n].
    Input:
        * The arrays 'xs' and 'ys'. 
        * A parameter 'freq' used to print only one in every freq line. The
          first, second and last line are always printed. If freq == 0, then
          only these lines are printed.
    Output: None.
    Prints: A header and the table described above.
    """
    def print_header():
        """
        Prints the table's header.
        """
        print("\n|       n      ", end="")
        print("    x_n            ", end="")
        print("    f(x_n)      |")
        print("|=================================================|")
        
    def print_line(n, x, y):
        """
        Pretty-prints n, x and y.
        """
        if n == 0:
            print(f"|       a", end="")
        elif n == 1:
            print(f"|       b", end="")
        else:
            print(f"|      {n - 1:02}", end="")
        print(f"    {x:15.8f}", end="")
        print(f"    {y:15.8f}   |")
    
    
    N = len(xs)
    if freq == 0:       # If freq == 0, print only first and last lines.
        freq = N - 1
    print_header()
    for n in range(0, 2):
        print_line(n, xs[n], ys[n])
    for n in range(2, N, freq):
        print_line(n, xs[n], ys[n])
    if n != N - 1:
        print_line(n, xs[N], ys[N])
    print("|_________________________________________________|\n")
        
    return None

In [26]:
def bisection_animation(f, a, b, N=4, title="", duration=0.75):
    """
    Displays an animation of the bissection method applied to a function.
    Input:
        * A continuous real function 'f'.
        * The two endpoitns 'a' and 'b' of an interval such that f(a)f(b) < 0.
        * The maximum number 'N' of iterations.
        * A title to be displayed at the top of the diagram.
        * The pause between slides of the animation, in seconds.
    Output:
        * Two lists xs and ys containing the estimates and the values of the
          function f at each of them.
    Displays:
        * The animation in a pop-up window.
    """
    import matplotlib.pyplot as plt
    import numpy as np
    
    
    def pause(duration):
        """
        Pauses the animation for duration seconds, provided that duration > 0.
        """
        if duration > 0:
            plt.pause(duration)

            
    def iterate(a, b):
        """
        Applies a single step of the bisection method to the interval [a, b].
        Returns its midpoint and the left, right endpoints of the next interval.
        """
        m = 0.5 * (a + b)                      # Midpoint of [a, b].
        if np.sign(f(a)) != np.sign(f(m)):     # [a, m] contains a zero.
            return m, a, m
        else:                                  # [m, b] contains a zero.
            return m, m, b
    
    
    P = 200                                    # Number of points in each plot.
    width = 1.75                               # Line width.
    marker_size = 5
    domain = np.linspace(a, b, P)              # Generates P nodes from a to b.
    xs = [a, b]                                # Stores the estimates.
    for _ in range(N):                         # Filling xs.
        m, a, b = iterate(a, b)
        xs.append(m)
    ys = [f(x) for x in xs]                    # Stores f of the estimates.
    # Lists containing the x and y coordinates for plotting vertical lines:
    xs_vert = [np.linspace(xs[n], xs[n], P) for n in range(N + 2)]
    ys_vert = [np.linspace(0, ys[n], P) for n in range(N + 2)]
    
    # Drawing the graph of f:
    plt.axhline(y=0.0, color='black', linestyle='-', lw=width)
    plt.xlabel("$ x $-axis")
    plt.ylabel("$ y $-axis")
    plt.title(title)
    plt.grid(True)
    plt.plot(domain, f(domain), label="$ y = f(x) $", lw=width)
    plt.legend()

    # Mark a on the x-axis and draw the vertical line x = a:
    pause(duration)
    plt.plot(xs[0], 0, color='black', marker="|", mew=width)
    pause(duration)
    plt.plot(xs_vert[0], ys_vert[0], linestyle='-', lw=width, label='$ a $')
    plt.plot(xs[0], 0, color='black', marker="|", mew=width)
    plt.plot(xs[0], ys[0], color='black', marker="o", ms=marker_size)
    plt.legend()
    
    # Mark b on the x-axis and draw the vertical line x = b:
    pause(duration)
    plt.plot(xs[1], 0, color='black', marker="|", mew=width)
    pause(duration)
    plt.plot(xs_vert[1], ys_vert[1], linestyle='-', lw=width, label='$ b $')
    plt.plot(xs[1], 0, color='black', marker="|", mew=width)
    plt.plot(xs[1], ys[1], color='black', marker="o", ms=marker_size)
    plt.legend()
    
    for n in range(2, N + 2):
        pause(duration)
        # Mark x_n on the x-axis:
        plt.plot(xs[n], 0, color='black', marker="|", mew=width)
        pause(duration)
        # Draw the vertical line x = x_n and plot (x_n, y_n):
        plt.plot(xs_vert[n], ys_vert[n], linestyle='-',
                 lw=width, label=f'$ x_{n - 1} = m_{n - 1} $')
        plt.plot(xs[n], ys[n], color='black', marker="o", ms=marker_size)
        plt.plot(xs[n], 0, color='black', marker="|", mew=width)
    plt.legend()
    
    return xs, ys

In [17]:
a = 0           # Extremidade esquerda do intervalo inicial, onde f vale -3.
b = 2           # Extremidade direita, onde f vale 3.
N = 5           # Número de iterações desejado.
pausa = 0.75    # Intervalo de tempo entre cada passo da animação, em segundos.
f = lambda x: x**3 - x - 3    # Função à qual o método será aplicado.
# Título a ser exibido no topo do diagrama:
titulo = "Método da bissecção para $ y = x^3 - x - 3,\ a = 0,\ b = 2 $."

xs, ys = bisection_animation(f, a, b, N, titulo, pausa)

![Exemplo do método da bissecção](fig_2-3_exemplo_1.png "Exemplo de aplicação do método da bissecção")

In [9]:
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|       a         0.00000000        -3.00000000   |
|       b         2.00000000         3.00000000   |
|      01         1.00000000        -3.00000000   |
|      02         1.50000000        -1.12500000   |
|      03         1.75000000         0.60937500   |
|      04         1.62500000        -0.33398438   |
|      05         1.68750000         0.11791992   |
|_________________________________________________|



## $ \S 3 $ Análise do desempenho do método da bissecção

Cada passo do método da bissecção corta o comprimento do intervalo anterior pela metade. Portanto dez passos reduzem o comprimento do intervalo original por um fator de $ 2^{10} > 1\,000 $. Mais geralmente, temos o seguinte resultado.

**Teorema 3.1 (análise do método da bissecção):** _Pelo método da bissecção, o número de iterações necessário para se localizar um zero dentro do intervalo inicial $ [a, b] $ com erro menor que $ \eps > 0 $ é dado por:_
\begin{equation*}\label{E:1}
\boxed{\ceil{\lg\bigg(\frac{b-a}{\eps}\bigg)} \quad \text{onde $ \lg = \log_2 $}} \tag{1}
\end{equation*}

**Prova:** Começando com o intervalo $ [a,b] $ original (primeiro passo),
*sabemos que em cada iteração há um zero dentro do intervalo atual. Após o $ n
*$-ésimo passo, o comprimento de $ [a, b] $ terá sido reduzido pelo fator de $
*2^{n-1} $. Mas como o resultado do método é o ponto méd io do intervalo atual,
*o que importa é a *metade* do seu comprimento, já que
esta é uma cota superior para a distância do ponto médio a um zero. Logo $ n $
deve ser grande o suficiente de modo que
$$
    \frac{b-a}{2^n} < \eps, \quad \text{ou equivalentemente,} \quad 
    n > \lg \left( \frac{b-a}{\eps} \right).
$$
O menor inteiro que satisfaz esta desigualdade é o teto do valor à direita, como em \eqref{E:1}. $\  \blacksquare $

📝 Informalmente, o resultado acima implica que para cada dígito adicional de precisão, precisamos de $ \lg 10 \approx 3.32 $ iterações a mais. Este desempenho deve ser comparado com o da busca incremental, que dependia _linearmente_ de $ \frac{b-a}{\eps} $; portanto para melhorar a precisão desta por um dígito decimal, precisamos _multiplicar_ o número de iterações por $ 10 $.

## $ \S 4 $ Critérios de parada e observações

Os critérios de parada que podemos utilizar são essencialmente os mesmos em todos os métodos para localização de zeros:

1. A distância da estimativa atual a um zero é menor que um $ \eps > 0 $ pré-escolhido.
2. O módulo do valor da função na estimativa atual é menor que um $ \delta > 0 $ escolhido previamente.
3. O número de iterações excede uma cota prefixada.

Além destas, podemos também considerar versões _relativas_ de 1 e 2, por exemplo:

4. O módulo do valor da função na estimativa $ m $ atual é menor que um $ \delta > 0 $ relativamente aos valores originais $ f(a) $ e $ f(b) $, i.e.:
$$
    \frac{\abs{f(m)}}{\min\left\{\abs{f(a)}\,,\,\abs{f(b)}\right\}} < \delta.
$$

Na implementação abaixo, o procedimento termina assim que algum dos critérios 1 ou 3 for satisfeito.

⚠️ Para que possamos sequer aplicar o método da bissecção a uma função, antes é necessário ter obtido um intervalo onde ela troca de sinal. Este requerimento preliminar geralmente pode ser satisfeito com a inspeção do gráfico da função ou com uma busca incremental (conforme explicado no caderno anterior).

⚠️ Por definição, $ a $ é um **pólo** de ordem $ m $ de uma função $ f $ se ele é um zero de multiplicidade $ m $ de $ 1 / f(x) $. Se $ f $ tiver um pólo de ordem ímpar num ponto, como o da função $ x \mapsto \frac{1}{x} $ em $ x = 0 $, então o método da bissecção poderá confundi-lo com um zero. Execute a animação abaixo para entender o que acontece nestes casos.

In [16]:
a = -0.15       # Extremidade esquerda do intervalo inicial.
b = 0.18        # Extremidade direita.
N = 5           # Número de iterações desejado.
pausa = 0.75    # Intervalo de tempo entre cada passo da animação, em segundos.
f = lambda x: 1 / x   # Função à qual o método será aplicado.
# Título a ser exibido no topo do diagrama:
titulo = "Método da bissecção para $ y = 1/x,\ a = -0.15,\ b = 0.18 $;\n"\
         "confundindo um pólo de ordem ímpar com um zero."

xs, ys = bisection_animation(f, a, b, N, titulo, pausa)

![Pólo de ordem ímpar](fig_2-3_exemplo_2.png "Método da bissecção confundindo um pólo de ordem ímpar com um zero")

In [13]:
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|       a        -0.15000000        -6.66666667   |
|       b         0.18000000         5.55555556   |
|      01         0.01500000        66.66666667   |
|      02        -0.06750000       -14.81481481   |
|      03        -0.02625000       -38.09523810   |
|      04        -0.00562500      -177.77777778   |
|      05         0.00468750       213.33333333   |
|_________________________________________________|



## $ \S 5 $ Implementação do método da bissecção

In [20]:
def bisection(f, a, b, eps, max_iter):
    """
    Uses the bisection method to approximate a zero of a function.
    Input:
        * A real continuous function 'f'.
        * Points 'a' and 'b' such that f(a)f(b) < 0 and f is defined on [a, b].
        * The maximum tolerance 'eps' for the error.
        * The maximum number 'max_iter' of iterations.
    Output:
        * Two lists, 'xs' and 'ys', containing the estimates and the value of f
          at each of them.
    Prints:
        * The last estimate, the value of f at this estimate, the number of
          iterations and an upper bound for the error.
    """
    from numpy import sign
    

    iterations = 0                        # Counts the number of iterations.
    f_a = f(a)                            # Storing the value of f at a.
    f_b = f(b)                            # Storing the value of f at b.
    xs = [a, b]                           # List to store the estimates.
    ys = [f_a, f_b]                       # List to store f of the estimates.
    if eps <= 0:                          # Error: invalid value for 'eps'.
        raise ValueError("The tolerance must be positive!")
    if f_a == 0:                          # a is a zero.
        print("a is an exact zero.")
        return a
    elif f_b == 0:                        # b is a zero.
        print("b is an exact zero.")
        return b
    elif sign(f_a) == sign(f_b):          # Error: cannot guarantee zero exists.
        raise ValueError("The function takes on the same sign"
                         "at the given endpoints!")
    
    while (b - a) >= eps and iterations <= max_iter:
        m = 0.5 * (a + b)                 # Midpoint of current interval.
        f_m = f(m)                        # Storing the value of f at m.
        xs.append(m)
        ys.append(f_m)
        if f_m == 0:
            print("Found an exact zero.")
            return m
        elif sign(f_a) != sign(f_m):      # [a, m] contains a zero.
            b = m                         # Take m to be the new b.
        else:                             # [m, b] contains a zero.
            a = m                         # Take m to be the new a.
            f_a = f_m
        iterations += 1
        
    print(f"Found an approximate zero:\n{m:15.9f}")
    print(f"after {iterations} iterations, with an error of at most {b - a}.")
    print(f"The value of f at this point is:\n{f(m):15.9f}")
    
    return xs, ys

## $ \S 6 $ Problemas

**Problema 1:**

(a) Estime $ \sqrt[3]{2} $ com $ 5 $ dígitos decimais de precisão usando o método da bissecção. *Dica:* Considere a função $ f(x) = x^3 - 2 $.

(b) Use o Teorema 3.1 para calcular o número de iterações necessárias.

*Solução:*

**Problema 2:** Seja $ y = \tan x $ a função tangente. Recorde que 
$$
\tan\left(\frac{\pi}{4}\right) = 1 \quad \text{e} \quad \tan\left(\frac{2\pi}{3}\right) = -\sqrt{3}
$$
Aplique o método da bissecção a $ f = \tan $ no intervalo $ \left[\frac{\pi}{4}\,,\,\frac{2\pi}{3}\right] $ e explique o resultado.

*Solução:*

**Problema 3:**

(a) Encontre a menor raiz real positiva da equação
$$
x^3 - 3.45x^2 - 5.72 x + 6.31 = 0
$$
com uma precisão melhor que $ 10^{-3} $ usando o método da bissecção.

(b) Considerando o intervalo original que você utilizou, quantos passos são necessários?

*Solução:*

**Problema 4:** Obtenha uma estimativa para $ \pi $, correta até a sexta casa decimal, utilizando o método da bissecção para aproximar o primeiro zero positivo da função seno. 

*Solução:*

**Problema 5:**

(a) Mostre que a equação $ xe^x = 1 $ possui uma única raiz em $ \mathbb{R} $. *Dica:* Calcule a derivada da função $ f(x) = xe^x - 1 $.

(b) Mostre que esta raiz se encontra no intervalo $ [0.5, 1] $.

(c) Estime esta raiz com erro máximo de $ 10^{-5} $, usando o método da bissecção.

(d) Quantos passos são necessários para garantir esta precisão?

*Solução:*

**Problema 6:** Resolva o exercício anterior para a equação $ x^2 + \ln x = 0 $ ($ x > 0 $).

*Solução:*

**Problema 7:** Encontre uma raiz de cada uma das equações abaixo com erro $ < 10^{-3} $. *Dica:* Antes de aplicar o método da bissecção, é necessário encaixotar um zero. Para isto, utilize a análise gráfica ou a busca incremental. Os procedimentos correspondentes estão na [última seção](#auxiliar) deste caderno.

(a) $ x \ln x = 1 $.

(b) $ \cos x = x^2 $.

(c) $ x^5 - 3x^4 - 6x^3 + 4x^2 + 5x - 3 = 0 $.

(d) $ \tan x = x + 2 e^x $.

(e) $ \cos x \cosh x = 1 $, onde por definição $ \cosh x = \frac{e^x + e^{-x}}{2} $.

*Solução:*

## $ \S 7 $ ⚡ Análise da convergência do método da bissecção

**Teorema 7.1:** _Seja $ f \colon [a, b] $ uma função contínua 
tal que $ f(a) $ e $ f(b) $ têm sinais opostos. Seja $ [a_n, b_n] $ o intervalo
resultante após a $ n $-ésima iteração, com $ [a_0, b_0] = [a, b] $. Então ou o
método da bissecção produz um zero exato após um número finito de passos, ou as
seqüências $ (a_n) $ e $ (b_n) $ construídas por ele convergem a um mesmo zero de
$ f $, que pertence a todos os intervalos $ [a_n, b_n] $._

**Prova:** Observe que
$$
a_0 \le a_1 \le a_2 \le \cdots a_n \le \cdots \le \cdots \le b_n \le \cdots b_2 \le b_1 \le b_0
$$
e $ (b_n - a_n) \to 0 $. Mais detalhadamente, por construção valem:
1. A seqüência $ (a_n) $ é crescente e limitada superiormente, já que $
a_n \le  b_0 $ para todo $ n $. 
2. A seqüência $ (b_n) $ é decrescente e limitada inferiormente, já que $ a_0
\le b_n $ para todo $ n $. 
3. $ a_n \le b_n $ e $ b_n - a_n = \frac{(b - a)}{2^n} $ para todo $ n $.

Pela *completude* dos números reais, qualquer seqüência crescente e
limitada superiormente converge, e o limite é a _menor cota superior_ do 
conjunto de valores desta seqüência. Analogamente, qualquer seqüência
decrescente e limitada inferiormente converge à _maior cota inferior_ do
conjunto de valores dela.

Seja $ \zeta $ o limite de $ a_n $. Então, como 
$$
b_n - a_n = \frac{b - a}{2^{n}}
$$
e como $ (b_n) $ também converge, deduzimos que o limite desta também é $ \zeta
$. Finalmente, fazendo $ n \to \infty $ na desigualdade
$$
f(a_n) f(b_n) < 0
$$
e usando a continuidade de $ f $, concluímos que
$$
\lim_n \big[f(a_n) f(b_n)] = f(\lim a_n) f(\lim b_n) = f(\zeta) f(\zeta) =
f(\zeta) ^2 \le 0, .
$$
Isto só é possível se $ f(\zeta) = 0 $, ou seja, se $ \zeta $ é um zero de $ f
$. $\ \ \blacksquare $

## $ \S 8 $ Procedimentos auxiliares<a name="auxiliar"></a>

In [25]:
def incremental_search(f, a, b, h):
    """
    Beginning with x_0 = a and x_1 = a + h and successively incrementing (or
    decrementing) both by h, returns the first pair of consecutive nodes where f
    takes on opposite signs.
    """
    from numpy import sign
    

    if h == 0:
        raise ValueError("0 is not a valid value for h!")
    # Initializing:
    x_0 = a
    x_1 = a + h
    f_0 = f(x_0)
    f_1 = f(x_1)
    
    while sign(f_0) == sign(f_1):
        if x_1 > b:
            return None, None
        x_0, f_0 = x_1, f_1
        x_1 += h
        f_1 = f(x_1)
        
    return x_0, x_1

In [ ]:
def plot_functions(a, b, N, *fs):
    """
    Input:
        * The left and right endpoints 'a' and 'b' of the plot interval.
        * The number 'N' of points used in each plot.
        * Any quantity (>= 1) of functions defined on [a, b].
    Output: None.
    Displays:
        * The graphs of the given functions in a single diagram.
    """
    import matplotlib.pyplot as plt
    import numpy as np
    
    
    xs = np.linspace(a, b, N)      # Create sample of N values of x in [a, b].
    for n, f in enumerate(fs):
        plt.plot(xs, f(xs), label=f'função {n}')
    plt.xlabel("$ x $-axis")
    plt.ylabel("$ y $-axis")
    plt.grid(True)
    plt.legend()

    return None